In [1]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
import random

from keras.models import load_model

model = load_model("weights.13-0.18.hdf5")
random.seed(3)

def find_vessel(name, vessels):
    for v in vessels:
        if v.startswith(name):
            return v
    return -1

def bgr2rgb(img):
    return img[...,::-1]

def load_color_img(path):
    return bgr2rgb(cv.imread(path))

def load_gray_img(path):
    return cv.imread(path, 0)

def pad_img(img, padsize):
    #rows
    if len(img.shape) == 3:
        upper_pad = img[img.shape[0]-padsize:img.shape[0], :, :]
        lower_pad = img[0:padsize+1, :, :]
    elif len(img.shape) == 2:
        upper_pad = img[img.shape[0]-padsize:img.shape[0], :]
        lower_pad = img[0:padsize+1, :]
    img = np.concatenate((upper_pad, img), axis=0)
    img = np.concatenate((img, lower_pad), axis=0)

    #columns
    if len(img.shape) == 3:
        p = np.zeros((img.shape[0], padsize, img.shape[2]), dtype=int)
    elif len(img.shape) == 2:
        p = np.zeros((img.shape[0], padsize), dtype=int)
    
    img = np.concatenate((p, img), axis=1)
    img = np.concatenate((img, p), axis=1)

    return img

def save_img(path, img):
    img = bgr2rgb(img)
    cv.imwrite(path, img)
    
def is_black(img):
    maxcolor = 255.0 * img.shape[0] * img.shape[1] * img.shape[2]
    if np.sum(img)/maxcolor <= 0.01:
        return True
    return False
    
sample_size = 33
pad_size = int( np.floor(sample_size/2.0) )

data_path = os.getcwd() + "/eye_data" + "/set1"
test_path = data_path + "/4testing" + "/"

test_imgs = [f for f in os.listdir(test_path) if f.lower().find(".jpg") != -1]
test_vessels = [f for f in os.listdir(test_path) if f.lower().find(".tif") != -1]

for t in test_imgs:
    name = t[:t.lower().find(".jpg")]
    v = find_vessel(name, test_vessels)

    eye_img = load_color_img(test_path+t)
    vessel_img = load_gray_img(test_path+v)

    eye_padded = pad_img(eye_img, pad_size)
    vessel_padded = pad_img(vessel_img, pad_size)
    
    vessels_predicted_img = np.zeros(vessel_img.shape, dtype=np.float32)
    samples = [
        [eye_padded[x-pad_size:x+pad_size+1, y-pad_size:y+pad_size+1] for y in range(pad_size, eye_padded.shape[1]-pad_size)]
              for x in range(pad_size, eye_padded.shape[0]-pad_size-1)]
    print(vessel_img.shape)
    print(len(samples), len(samples[0]))
    
    plt.imshow(eye_img)
    plt.show()
    plt.imshow(vessel_img)
    plt.show()
    
    print("prediction start...")
    for idx, line in enumerate(samples):
        line = np.array(line, dtype=np.float32)
        predicted_classes = model.predict(line)
        predicted_classes = np.array(list(map(lambda x : np.argmax(x), predicted_classes))) * 255.0
        vessels_predicted_img[idx] = predicted_classes
        print(idx)
        if idx % 100 == 0:
            plt.imshow(vessels_predicted_img)
            plt.show()
    print("predicted!")

    plt.imshow(vessels_predicted_img)
    plt.show()
    
    break

   
    



Using TensorFlow backend.


MemoryError: 

In [9]:
test_imgs


['04_g.jpg', '06_dr.JPG', '11_h.jpg', '13_dr.JPG', '14_h.jpg']

In [5]:
import sys
sys.version

'3.5.4 |Anaconda custom (64-bit)| (default, Nov  8 2017, 14:34:30) [MSC v.1900 64 bit (AMD64)]'